In [8]:
import pandas as pd
import numpy as np
from glob import glob
import re
import unicodedata
# import mojimoji


In [9]:
path_list = glob('../Data/Original/*')
# path_list


In [10]:
# 元データ読み込み
# original_df_list = [pd.read_excel(path, engine='openpyxl') for path in path_list]
original_df = pd.concat([pd.read_excel(path, engine='openpyxl') for path in path_list], axis='index').iloc[:, [1, 3]]
# original_df


In [11]:
# 3ケタ
digit3_df = original_df.copy()
digit3_df = digit3_df[(digit3_df['記号'].astype(str).str.len() <= 4)\
        & (digit3_df['記号'].astype(str).str.len() != 1)].fillna(np.nan)
digit3_df['記号'] = digit3_df['記号'].replace('＜注＞', np.nan).ffill()
digit3_df = digit3_df[digit3_df['記号'].astype(str).str.len()==3]\
                     .reset_index(drop=True)
digit3_df['タイトル'] = digit3_df['タイトル'].astype(str).str.replace('\u3000|\t|\n', '')
digit3_df = digit3_df.groupby('記号').agg({'タイトル': '\n'.join}).reset_index()
digit3_df
digit3_df.to_csv('../Data/Dealed/ipc_3digit.csv', sep=',', encoding='utf-8', index=False)


In [17]:
# 4ケタ
df = original_df.copy().dropna(axis='index')
df = df[df['記号'].str.contains(r'^[A-Z]{1}\d{2}[A-Z]{1}$')].reset_index(drop=True)\
                  .rename(columns={'記号':'class', 'タイトル':'class_jp'})
for ind in range(len(df)):
    df.loc[ind, 'class_jp'] = unicodedata.normalize('NFKC', df.loc[ind, 'class_jp'])
df
for s in ['\u3000', '\t', '\n', r'']:
    df['class_jp'] = df['class_jp'].str.replace(s, '')
display(df.head())
display(df.info())


,class,class_jp
0,A01B,"農業または林業における土作業:農業機械または器具の部品,細部または附属具一般(播種,植え付け..."
1,A01C,"植付け;播種;施肥(農業機械または器具の部品,細部または付属具一般A01B51/00~A01..."
2,A01D,収穫;草刈り
3,A01F,"脱穀(コンバインA01D41/00) ;わら,乾草または類似のものの梱包;わら,乾草または類..."
4,A01G,"園芸;野菜,花,稲,果樹,ぶどう,ホップ,または海草の栽培;林業;灌水(果実,野菜,ホップま..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   class     651 non-null    object
 1   class_jp  651 non-null    object
dtypes: object(2)
memory usage: 10.3+ KB


None

In [13]:
for ind in range(len(df)):
    df.loc[ind, 'class_jp'] = unicodedata.normalize('NFKC', df.loc[ind, 'class_jp'])
df

,class,class_jp
0,A01B,"農業または林業における土作業:農業機械または器具の部品,細部または附属具一般(播種,植え付け..."
1,A01C,"植付け;播種;施肥(農業機械または器具の部品,細部または付属具一般A01B51/00~A01..."
2,A01D,収穫;草刈り
3,A01F,"脱穀(コンバインA01D41/00);わら,乾草または類似のものの梱包;わら,乾草または類似..."
4,A01G,"園芸;野菜,花,稲,果樹,ぶどう,ホップ,または海草の栽培;林業;灌水(果実,野菜,ホップま..."
...,...,...
646,H05K,"印刷回路;電気装置の箱体または構造的細部,電気部品の組立体の製造"
647,H10B,電子記憶装置[2023.01]
648,H10K,有機電気的固体装置[2023.01]
649,H10N,他に分類されない電気的固体装置[2023.01]


In [14]:
# df[df['class_jp'].str.contains(r'[０-９]')]

for ind in range(len(df)):
    df.loc[ind, 'class_jp'] = mojimoji.zen_to_han(df.loc[ind, 'class_jp'], kana=False, ascii=False, digit=True)
df

NameError: name 'mojimoji' is not defined

In [47]:
df['class_jp'].unique()

array(['農業または林業における土作業：農業機械または器具の部品，細部または附属具一般（播種，植え付けまたは施肥のための条溝または孔の形成又は覆土Ａ０１Ｃ５／００；根菜作物の収穫機械Ａ０１Ｄ；土作業装置に転換できる，または土作業ができる刈取機Ａ０１Ｄ４２／０４；土作業器具と結合した刈取機Ａ０１Ｄ４３／１２；土木工事のための土作業Ｅ０１，Ｅ０２，Ｅ２１）',
       '植付け；播種；施肥（農業機械または器具の部品，細部または付属具一般Ａ０１Ｂ５１／００～Ａ０１Ｂ７５／００）［２００６．０１］',
       '収穫；草刈り',
       '脱穀（コンバインＡ０１Ｄ４１／００）；わら，乾草または類似のものの梱包；わら，乾草または類似のものを束にする，またはしばるための定置装置または手道具；わら，乾草または類似のものの切断；農業収穫物または園芸収穫物の貯蔵（収穫物の集積を作りまたは配置するための装置Ａ０１Ｄ８５／００）',
       '園芸；野菜，花，稲，果樹，ぶどう，ホップ，または海草の栽培；林業；灌水（果実，野菜，ホップまたは類似のものの摘み取りＡ０１Ｄ４６／００；単細胞藻類培養Ｃ１２Ｎ１／１２）',
       '新規植物またはそれらを得るための処理；組織培養技術による植物の増殖',
       '酪農製品の製造（化学的事項に関してはＡ２３Ｃ参照）［２００６．０１］',
       '畜産；鳥，魚，昆虫の飼育；漁業；他に分類されない動物の飼育または繁殖；新規な動物', '動物の装蹄',
       '動物の捕獲，わな猟，または威嚇（分蜂捕獲または雄蜂捕獲の用具Ａ０１Ｋ５７／００；漁撈Ａ０１Ｋ６９／００～Ａ０１Ｋ９７／００；殺生物剤，有害生物忌避剤または誘因剤Ａ０１Ｎ）；有害な動物又は有害な植物の駆除装置',
       '人間または動物または植物の本体またはその一部の保存（食品または食料品の保存Ａ２３）；殺生物剤，例．殺菌剤，殺虫剤または除草剤として（望ましくない生物の殺除あるいは成長または増殖を防止する医療用の製剤，歯科用の製剤又は化粧品Ａ６１Ｋ）；有害生物忌避剤または誘引剤；植物生長調節剤',
       '化合物または組成物の殺生物，有害生物忌避，有害生物誘引または植物生長調節活性［８］',
       'ベイカー用焼がま；

In [8]:
df.to_csv('../Data/Dealed/IPC_class4digit.csv', sep=',', encoding='utf-8')